In [50]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.phrases import Phrases, Phraser
import gensim.downloader
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\18moh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Movie Dataset

In [51]:
ratings = pd.read_csv('data/ratings.csv', sep=',', encoding='latin-1')
movies = pd.read_csv('data/movies.csv', sep=',', encoding='latin-1')

In [52]:
data = movies.merge(ratings)
data.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [64]:
grouped_df = data.groupby("title").mean()
grouped_df = grouped_df[['movieId', 'rating']]

In [65]:
df = data[['movieId', 'title', 'genres']]
df = df.drop_duplicates()
df = df.reset_index()[df.columns]
df = df.merge(grouped_df)
df.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143
4,5,Father of the Bride Part II (1995),Comedy,3.071429


In [66]:
df['Desc'] = df['title'] + ' ' + df['genres']
df.head()

,movieId,title,genres,rating,Desc
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,Toy Story (1995) Adventure|Animation|Children|...
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,Jumanji (1995) Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,Grumpier Old Men (1995) Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143,Waiting to Exhale (1995) Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy,3.071429,Father of the Bride Part II (1995) Comedy


# Book Dataset

In [116]:
ratings = pd.read_csv('archive/ratings.csv', sep=',', encoding='latin-1')
books = pd.read_csv('archive/books.csv', sep=',', encoding='latin-1')

In [117]:
data = books.merge(ratings)
data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [118]:
grouped_df = data[['ISBN', 'Book-Rating']].groupby("ISBN").mean()
grouped_df = grouped_df.reset_index()
# grouped_df = grouped_df[['ISBN', 'Book-Rating']]

In [119]:
df = data[['ISBN', 'Book-Title', 'Book-Author']]
df['title'] = df['Book-Title']
df = df[['ISBN', 'title', 'Book-Author']]
df = df.drop_duplicates()
df = df.reset_index()[df.columns]
df = df.merge(grouped_df)
df = df.dropna()
df.head()

,ISBN,title,Book-Author,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,0.000000


In [120]:
df['Desc'] = df['title'] + ' ' + df['Book-Author']
df.head()

,ISBN,title,Book-Author,Book-Rating,Desc
0,0195153448,Classical Mythology,Mark P. O. Morford,0.000000,Classical Mythology Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright,4.928571,Clara Callan Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este,5.000000,Decision in Normandy Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,4.272727,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,0.000000,The Mummies of Urumchi E. J. W. Barber


In [121]:
for x in df.Desc:
    if(type(x) != str):
        print(x)

# Building the Recommender

In [122]:
#https://www.kdnuggets.com/2020/08/content-based-recommendation-system-word-embeddings.html

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df['cleaned'] = df['Desc'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)
df.head()

,ISBN,title,Book-Author,Book-Rating,Desc,cleaned
0,0195153448,Classical Mythology,Mark P. O. Morford,0.000000,Classical Mythology Mark P. O. Morford,classical mythology mark p o morford
1,0002005018,Clara Callan,Richard Bruce Wright,4.928571,Clara Callan Richard Bruce Wright,clara callan richard bruce wright
2,0060973129,Decision in Normandy,Carlo D'Este,5.000000,Decision in Normandy Carlo D'Este,decision normandy carlo d este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,4.272727,Flu: The Story of the Great Influenza Pandemic...,flu story great influenza pandemic 1918 search...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,0.000000,The Mummies of Urumchi E. J. W. Barber,mummies urumchi e j w barber


In [68]:
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [71]:
# Training our corpus with Google Pretrained Model

model = Word2Vec(vector_size = 420, window=5, min_count = 2, workers = -1)
model.build_vocab(corpus)

In [72]:
model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

(0, 0)

# Step 2

In [73]:
def vectors(x):
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    # Reading the each book description 
    for line in df['cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.key_to_index:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model.wv[word]
                else:
                    avgword2vec = avgword2vec + google_model.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)

In [89]:
# Recommending the Top 5 similar movies

def recommendations(title, number):
    
    # Calling the function vectors

    vectors(df)
    
    # finding cosine similarity for the vectors

    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)

    # taking the title and book image link and store in new data frame called books
    movies = df[['title']]
    #Reverse mapping of the index
    indices = pd.Series(df.index, index = df['title']).drop_duplicates()
         
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:number+1]
    movie_indices = [i[0] for i in sim_scores]
    recommend = movies.iloc[movie_indices]
    for index, row in recommend.iterrows():
        print(row['title'])

In [95]:
recommendations('Saving Private Ryan (1998)', 10)

Legionnaire (1998)
Savior (1998)
Windtalkers (2002)
Von Ryan's Express (1965)
Braveheart (1995)
Civil Action, A (1998)
Monument Ave. (1998)
54 (1998)
Rounders (1998)
Stepmom (1998)


In [45]:
vectors(df)

# Evaluation

In [ ]:
from tqdm import tqdm
def rankedRetrieval(actual,predicted,k):
    
    lenPred = len(predicted)
    if(len(predicted) > k):
        predicted=predicted[:k]
    
    count=0
    for value in predicted:
        if(value in actual):
            count+=1
    
    precision = count/len(actual)
    recall = count/lenPred
    
    return precision,recall

def apk(actual, predicted, k):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted,k):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])


def computeScore(train,test,numRecommendations):

    avgPrecisionBook = []
    avgRecallBook = []
    avgPrecisionAuthor = []
    avgRecallAuthor= []

    recommended_movies = np.array([])
    recommended_authors = np.array([])

    
    for book in train.title.values:
        recommendation_output = book_recommendations(book, cosine_sim_df, data[['title', 'author']],numRecommendations)
        recommended_books=np.concatenate((recommended_books,
                                           np.array(recommendation_output.title.values)
                                          ),axis=0)
        recommended_authors=np.concatenate((recommended_authors,
                                   np.array(recommendation_output.author.values)
                                  ),axis=0)
        precisionBook,recallBook = rankedRetrieval(list(test.title.values),list(recommendation_output.title.values),numRecommendations)
        precisionAuthor,recallAuthor = rankedRetrieval(list(test.author.values),list(recommendation_output.author.values),numRecommendations)
        
        avgPrecisionBook.append(precisionBook)
        avgRecallBook.append(recallBook)
        
        avgPrecisionAuthor.append(precisionAuthor)
        avgRecallAuthor.append(recallAuthor)
        
        mapk_book = mapk(list(test.title.values),list(recommended_books),numRecommendations)
        mapk_author = mapk(list(test.author.values),list(recommended_author),numRecommendations)
    
    return np.mean(avgPrecisionBook),np.mean(avgRecallBook),mapk_book,\
                np.mean(avgPrecisionAuthor),np.mean(avgRecallAuthor),mapk_author

In [ ]:
def evaluation(test_split=0.2,numRecommendations=10):

    
    avgPrecisionMovie = []
    avgRecallMovie = []
    avgPrecisionGenre = []
    avgRecallGenre = []
    
    mapk_movie_scores = []
    mapk_genre_scores = []
    
    test_length=[]
    
    for usrId in tqdm(np.random.choice(ratings.user_id.unique(),100, replace=False)): 
        sample_data = rating_movie_merged[rating_movie_merged['user_id']==usrId].reset_index().drop('index',axis=1)
        sample_data = sample_data.sort_values('rating',ascending=False)
        data_split_length = int(len(sample_data)*test_split)
        
        
        try:
            train = sample_data[sample_data['rating'] > 2].sample(data_split_length).copy()
            
        except:
            continue
            print('Failed! userID:{}'.format(usrId))
        
        test = sample_data.drop(index=train.index).copy()
        

        if(len(test) < 2):
            continue

            
        precisionMovie,recallMovie,mapkMovie,precisionGenre,recallGenre,mapkGenre = computeScore(train,test,numRecommendations)
        avgPrecisionMovie.append(precisionMovie)
        avgRecallMovie.append(recallMovie)
        
        avgPrecisionGenre.append(precisionGenre)
        avgRecallGenre.append(recallGenre)
        
        
        mapk_movie_scores.append(mapkMovie)
        mapk_genre_scores.append(mapkGenre)
        
        test_length.append(len(test))
        
    return np.array(avgPrecisionMovie),np.array(avgRecallMovie),np.array(mapk_movie_scores)\
                    ,np.array(avgPrecisionGenre),np.array(avgRecallGenre),np.array(mapk_genre_scores)\
                    ,np.array(test_length)

In [ ]:
pm_list = []
rm_list = []
pg_list = []
rg_list = []
mapkM_list=[]
mapG_list=[]
for i in range(10):
    pm,rm,mapkM,pg,rg,mapkG,len_test = evaluation()
    pm_list.append(np.mean(pm))
    rm_list.append(np.mean(rm))
    mapkM_list.append(mapkM)
    
    pg_list.append(np.mean(pg))
    rg_list.append(np.mean(rg))
    mapG_list.append(mapkG)

In [ ]:
print("mean:{:.3f} | STD:{:.3f}".format(np.mean(pm_list),np.std(pm_list)))
print("mean:{:.3f} | STD:{:.3f}".format(np.mean(rm_list),np.std(rm_list)))
print("mean:{:.3f} | STD:{:.3f}".format(np.mean(mapkM),np.std(mapkM)))

print('------------------------')

print("mean:{:.3f} | STD:{:.3f}".format(np.mean(pg_list),np.std(pg_list)))
print("mean:{:.3f} | STD:{:.3f}".format(np.mean(rg_list),np.std(rg_list)))
print("mean:{:.3f} | STD:{:.3f}".format(np.mean(mapkG),np.std(mapkG)))